In [1]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Helper libraries
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import math

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large, ResNet152
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import model_from_json


2023-03-31 16:52:17.305428: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 16:52:17.544364: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-31 16:52:18.795211: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/
2023-03-31 16:52:18.795357: W t

Num GPUs Available:  1


## Data Splitting ##

In [7]:
# df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
# df['filename'] = df["image_id"].astype(str) + ".png"
# df = df.sample(frac=1).reset_index(drop=True)
# df.to_csv("training_norm_shuffle.csv", encoding='utf-8', index=False)

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm_shuffle.csv')
print(df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0           414  0.8125    1.0    414.png
1         17532  0.6875    1.0  17532.png
2         16480  0.3125    0.0  16480.png
3          6450  0.6875    0.0   6450.png
4         12314  0.6250    0.0  12314.png
...         ...     ...    ...        ...
14821      9844  0.6875    1.0   9844.png
14822     12367  0.5000    1.0  12367.png
14823       343  0.9375    1.0    343.png
14824      3838  0.6875    0.0   3838.png
14825     13301  0.5000    1.0  13301.png

[14826 rows x 4 columns]


## Data Preprocessing ##

In [8]:
batch_size = 32
img_size = [240, 320]
shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift,
    brightness_range=(0.75, 1.25)
    
)

speed_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

angle_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

speed_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

speed_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    shuffle=False,
    class_mode='other')



Found 8895 validated image filenames.
Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.
Found 2966 validated image filenames.


In [9]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)
submission_datagen = ImageDataGenerator(rescale = 1./255)

submission_images = submission_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.


## Transfer Learning ##

In [10]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-1]:
    layer.trainable = False

## Model Creation and Evaluation ##

In [32]:
# Speed CNN base
speed_model = models.Sequential()

speed_model.add(transfer)
speed_model.build()
        
speed_model.add(layers.Conv2D(256, (3, 3), strides=(2,2), activation='relu'))
speed_model.add(layers.BatchNormalization())

speed_model.add(layers.Flatten()),

# speed_model.add(layers.Dense(128, activation='relu'))
# speed_model.add(layers.BatchNormalization())
# speed_model.add(layers.Dropout(0.2))

speed_model.add(layers.Dense(64, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

# Output layer
speed_model.add(layers.Flatten())
speed_model.add(layers.Dense(1,  activation='sigmoid'))

speed_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 4, 256)         2949376   
                                                                 
 batch_normalization_8 (Batc  (None, 3, 4, 256)        1024      
 hNormalization)                                                 
                                                                 
 flatten_8 (Flatten)         (None, 3072)              0         
                                                                 
 dense_8 (Dense)             (None, 64)                196672    
                                                                 
 batch_normalization_9 (Batc  (None, 64)              

In [33]:
# Angle CNN base
angle_model = models.Sequential()

angle_model.add(transfer)
angle_model.build()
        
angle_model.add(layers.Conv2D(256, (3, 3), strides=(2,2), activation='relu'))
angle_model.add(layers.BatchNormalization())

angle_model.add(layers.Flatten()),

# model_1.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.Dense(64, activation='relu'))
angle_model.add(layers.BatchNormalization())
angle_model.add(layers.Dropout(0.2))

# Output layer
angle_model.add(layers.Flatten())
angle_model.add(layers.Dense(1,  activation='relu'))

angle_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 4, 64)          737344    
                                                                 
 batch_normalization_10 (Bat  (None, 3, 4, 64)         256       
 chNormalization)                                                
                                                                 
 flatten_10 (Flatten)        (None, 768)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                49216     
                                                                 
 batch_normalization_11 (Bat  (None, 64)              

In [34]:
#speed_model.build()
#angle_model.build()

In [35]:
speed_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

angle_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

In [36]:
history = speed_model.fit(
    speed_train_generator,
    batch_size=batch_size,
    validation_data=speed_val_generator,
    callbacks=[es],
    epochs=1
)

278/278 [==============================] - 109s 377ms/step - loss: 0.0414 - val_loss: 0.0242


In [37]:
test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)

93/93 [==============================] - 8s 90ms/step - loss: 0.0247


In [38]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100
278/278 [==============================] - 113s 395ms/step - loss: 0.3157 - val_loss: 0.1431
Epoch 2/100
278/278 [==============================] - 102s 366ms/step - loss: 0.1442 - val_loss: 0.0676
Epoch 3/100
278/278 [==============================] - 108s 390ms/step - loss: 0.0773 - val_loss: 0.0412
Epoch 4/100
278/278 [==============================] - 116s 416ms/step - loss: 0.0461 - val_loss: 0.0235
Epoch 5/100
278/278 [==============================] - 118s 424ms/step - loss: 0.0289 - val_loss: 0.0139
Epoch 6/100
278/278 [==============================] - 117s 422ms/step - loss: 0.0167 - val_loss: 0.0085
Epoch 7/100
278/278 [==============================] - 116s 418ms/step - loss: 0.0115 - val_loss: 0.0076
Epoch 8/100
278/278 [==============================] - 116s 419ms/step - loss: 0.0102 - val_loss: 0.0073
Epoch 9/100
278/278 [==============================] - 112s 403ms/step - loss: 0.0097 - val_loss: 0.0075
Epoch 10/100
278/278 [==============================] -

In [39]:
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

93/93 [==============================] - 10s 105ms/step - loss: 0.0087


In [40]:
#speed_model.save("speed-model.h5")
angle_model.save("angle-model.h5")

## Model Prediction ##

In [41]:
angle_prediction = angle_model.predict(submission_images)
speed_prediction = speed_model.predict(submission_images)

speed_prediction = speed_prediction.flatten()
angle_prediction = angle_prediction.flatten()

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('Submission.csv', index_label="image_id")


32/32 [==============================] - 4s 91ms/step


In [42]:
angle_prediction = angle_model.predict(x_evaluate)
speed_prediction = speed_model.predict(x_evaluate)

diff = abs(x_evaluate["angle"] - angle_prediction)
results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":angle_prediction, "Difference":diff})
#results

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":speed_prediction})
#results

## Fine-tuning ##

In [47]:
speed_model = load_model("angle-model.h5")
angle_model = load_model("angle-model.h5")


angle_transfer = angle_model.layers[0]
angle_transfer.trainable = True

for layer in angle_transfer.layers[:-25]:
    layer.trainable = False
        
angle_model.summary()


speed_transfer = speed_model.layers[0]
speed_transfer.trainable = True

for layer in speed_transfer.layers[:-25]:
    layer.trainable = False
    
speed_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 4, 256)         2949376   
                                                                 
 batch_normalization_6 (Batc  (None, 3, 4, 256)        1024      
 hNormalization)                                                 
                                                                 
 flatten_6 (Flatten)         (None, 3072)              0         
                                                                 
 dense_6 (Dense)             (None, 64)                196672    
                                                                 
 batch_normalization_7 (Batc  (None, 64)              

In [48]:

speed_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001),
    loss='mean_squared_error'
)

angle_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0005,
    patience=30,
    verbose=1,
    mode='min',
    restore_best_weights=True
)


In [50]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=300
)

Epoch 1/300
278/278 [==============================] - 107s 384ms/step - loss: 0.0060 - val_loss: 0.0061
Epoch 2/300
278/278 [==============================] - 107s 384ms/step - loss: 0.0059 - val_loss: 0.0061
Epoch 3/300
278/278 [==============================] - 107s 386ms/step - loss: 0.0058 - val_loss: 0.0061
Epoch 4/300
278/278 [==============================] - 107s 383ms/step - loss: 0.0057 - val_loss: 0.0061
Epoch 5/300
278/278 [==============================] - 111s 398ms/step - loss: 0.0056 - val_loss: 0.0061
Epoch 6/300
278/278 [==============================] - ETA: 0s - loss: 0.0056

: 

: 

In [ ]:
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

In [ ]:
history = speed_model.fit(
    speed_train_generator,
    batch_size=batch_size,
    validation_data=speed_val_generator,
    callbacks=[es],
    epochs=1
)

In [ ]:
from sklearn.metrics import classification_report

test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)


In [ ]:
speed_model.save("speed-model-fine-tuning.h5")
angle_model.save("angle-model-fine-tuning.h5")

In [ ]:
angle_prediction = angle_model.predict(submission_images)
speed_prediction = speed_model.predict(submission_images)

speed_prediction = speed_prediction.flatten()
angle_prediction = angle_prediction.flatten()

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('Submission-fine-tuning.csv', index_label="image_id")

In [ ]:
model = load_model("model-fine-tuning.hdf5")
plot_model(model, show_layer_names=True, show_shapes=True,show_layer_activations=True)

## Clean Image File ##

In [ ]:
speed_model = load_model("speed-model.h5")
angle_model = load_model("angle-model-fine-tuning.h5")

model = load_model("model.h5")

x_all = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
x_all['filename'] = x_all["image_id"].astype(str) + ".png"

clean_datagen = ImageDataGenerator(rescale = 1./255)

clean_generator = clean_datagen.flow_from_dataframe(
    dataframe=x_all,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

angle_prediction = angle_model.predict(clean_generator)
speed_prediction = speed_model.predict(clean_generator)

# predict = model.predict(clean_generator)

# angle_prediction = predict[:,0]
# speed_prediction = predict[:,1]

diff = abs(x_all["angle"] - angle_prediction)

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

correct = (x_all["speed"] == speed_prediction)

clean_data = {
    "image_id":x_all["image_id"], 
    "Actual Angle":x_all["angle"], 
    "Predicted Angle":angle_prediction, 
    "Angle Difference":diff, 
    "Actual Speed":x_all["speed"], 
    "Predicted Speed":speed_prediction,
    "Correct?":correct
}

df = pd.DataFrame(clean_data)
df.index += 1 
#df.to_csv('clean-results.csv', index_label="image_id")


## Run other models ##

In [51]:
#angle_model = load_model("angle-model-fine-tuning.h5")
best_speed_model = load_model("best-sakthi.hdf5")
best_angle_model = load_model("angle-model-fine-tuning.h5")
sakthi_model = load_model("sakthi.hdf5")

best_speed_prediction = best_speed_model.predict(submission_images)
best_angle_prediction = best_angle_model.predict(submission_images)
sakthi_prediction = sakthi_model.predict(submission_images)

# speed_prediction = speed_prediction[0]

best_speed_prediction = best_speed_prediction.flatten()
best_angle_prediction = best_angle_prediction.flatten()

# sakthi_prediction = sakthi_prediction.flatten()



sakthi_angle = sakthi_prediction[1]
sakthi_speed = sakthi_prediction[0]

sakthi_angle = sakthi_angle.flatten()
sakthi_speed = sakthi_speed.flatten()


sakthi_speed += 0.5
sakthi_speed = np.floor(sakthi_speed)

diff = abs(best_angle_prediction - sakthi_angle)

best_speed_prediction += 0.5
best_speed_prediction = np.floor(best_speed_prediction)

match = (best_speed_prediction == sakthi_speed)

# validate = {
#     "Best Angle":best_angle_prediction, 
#     "Sakthi Angle":sakthi_angle, 
#     "Angle Difference":diff, 
#     "Best Speed":best_speed_prediction, 
#     "Sakthi Speed":sakthi_speed,
#     "Match":match
# }
validate = {"angle":sakthi_angle, "speed":best_speed_prediction}


df = pd.DataFrame(validate)
df.index += 1 
df.to_csv('Submission-alt.csv', index_label="image_id")

32/32 [==============================] - 4s 127ms/step


In [42]:
speed_model = load_model("angle-model-fine-tuning.h5")

test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

93/93 [==============================] - 8s 81ms/step - loss: 0.0058


## Multi-Model Testing ##

In [13]:
def model_run(con:int, dense:int, batch: int, epoch:int):
    
    input_shape = [240, 320, 3]
    transfer = ResNet152(
        input_shape=input_shape, 
        include_top=False, 
        weights='imagenet',
        classifier_activation="relu"
    )

    for layer in transfer.layers[:-1]:
        layer.trainable = False
    
    # Angle CNN base
    angle_model = models.Sequential()

    angle_model.add(transfer)
    angle_model.build()
            
    angle_model.add(layers.Conv2D(con, (3, 3), strides=(2,2), activation='relu'))
    angle_model.add(layers.BatchNormalization())

    angle_model.add(layers.Flatten()),

    angle_model.add(layers.Dense(dense, activation='relu'))
    angle_model.add(layers.BatchNormalization())
    angle_model.add(layers.Dropout(0.2))

    # Output layer
    angle_model.add(layers.Flatten())
    angle_model.add(layers.Dense(1,  activation='relu'))

    ## Compile
    angle_model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.005),
        loss='mean_squared_error'
    )

    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.001,
        patience=10,
        verbose=1,
        mode='min',
        restore_best_weights=True
    )

    history = angle_model.fit(
        angle_train_generator,
        batch_size=batch,
        validation_data=angle_val_generator,
        callbacks=[es],
        epochs=epoch
    )
    


    return angle_model, history

def model_run1(batch: int, epoch:int, angle_model):
    ## Compile
    angle_model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error'
    )

    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode='min',
        restore_best_weights=True
    )

    history = angle_model.fit(
        angle_train_generator,
        batch_size=batch,
        validation_data=angle_val_generator,
        callbacks=[es],
        epochs=epoch
    )

    return angle_model, history

def model_fine_tuning(angle_model):
    
    angle_transfer = angle_model.layers[0]
    angle_transfer.trainable = True

    for layer in angle_transfer.layers[:-25]:
        layer.trainable = False
        
    angle_model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.00001),
        loss='mean_squared_error'
    )

    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.0005,
        patience=30,
        verbose=1,
        mode='min',
        restore_best_weights=True
    )
    
    history = angle_model.fit(
        angle_train_generator,
        batch_size=8,
        validation_data=angle_val_generator,
        callbacks=[es],
        epochs=300
    )
    
    return angle_model, history



In [14]:
conv = [128] #,32,64,128,256,512]
dense = [64] #[64,128,256,512]
for i in range(len(conv)):
    for j in range(len(dense)):

        angle_train_generator = training_datagen.flow_from_dataframe(
            dataframe=x_train,
            directory="machine-learning-in-science-ii-2023/training_data/training_data",
            x_col="filename",
            y_col="angle",
            target_size=img_size,
            batch_size=32,
            shuffle=False,
            class_mode='other')
        
        angle_model, history = model_run(conv[i], dense[j], 32, 100)
        
        loss = angle_model.evaluate(
            angle_eval_generator,
            verbose=1
        )
        
        angle_model.save("model_last.h5")
        
        
        angle_train_generator = training_datagen.flow_from_dataframe(
            dataframe=x_train,
            directory="machine-learning-in-science-ii-2023/training_data/training_data",
            x_col="filename",
            y_col="angle",
            target_size=img_size,
            batch_size=8,
            shuffle=False,
            class_mode='other')
        
        angle_model, history = model_run1(8, 100, angle_model)
        
        angle_model.save("model_last-2.h5")

        
        loss1 = angle_model.evaluate(
            angle_eval_generator,
            verbose=1
        )
        
        angle_model, history = model_fine_tuning(angle_model)
        
        extra_loss = angle_model.evaluate(
            angle_eval_generator,
            verbose=1
        )
        
        angle_model.save("model_last-3.h5")

        
        df = pd.DataFrame({"conv":conv[i], "dense":dense[j],"loss":loss, "loss1":loss1, "fine tuning":extra_loss}, index=[0])
        df.to_csv('results.csv', mode='a', header=False)



Found 8895 validated image filenames.
Epoch 1/100


2023-03-31 16:57:02.577025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-31 16:57:04.114185: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-03-31 16:57:06.550635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-31 16:57:06.577592: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x6c32610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-31 16:57:06.577630: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-03-31 16:57:06.595954: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-31 16:57:06.833593:

278/278 [==============================] - 146s 482ms/step - loss: 0.1297 - val_loss: 0.0133
Epoch 2/100
278/278 [==============================] - 132s 475ms/step - loss: 0.0149 - val_loss: 0.0097
Epoch 3/100
278/278 [==============================] - 131s 469ms/step - loss: 0.0125 - val_loss: 0.0087
Epoch 4/100
278/278 [==============================] - 134s 482ms/step - loss: 0.0113 - val_loss: 0.0104
Epoch 5/100
278/278 [==============================] - 130s 467ms/step - loss: 0.0105 - val_loss: 0.0102
Epoch 6/100
278/278 [==============================] - 128s 460ms/step - loss: 0.0102 - val_loss: 0.0378
Epoch 7/100
278/278 [==============================] - 124s 447ms/step - loss: 0.0099 - val_loss: 0.0206
Epoch 8/100
278/278 [==============================] - 122s 440ms/step - loss: 0.0095 - val_loss: 0.0118
Epoch 9/100
278/278 [==============================] - 122s 439ms/step - loss: 0.0093 - val_loss: 0.0230
Epoch 10/100
278/278 [==============================] - 123s 441ms/

## Retraining on new data ##

In [19]:
# df = pd.read_csv(r'machine-learning-in-science-ii-2023/new_data.csv')
# df['filename'] = df["image_id"].astype(str) + ".png"
# df = df.sample(frac=1).reset_index(drop=True)
# df.to_csv("new_data_shuffle.csv", encoding='utf-8', index=False)

df = pd.read_csv(r'machine-learning-in-science-ii-2023/new_data_shuffle.csv')
print(df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


      image_id   angle  speed   filename
0        19246  0.5625      1  19246.png
1        18040  0.5625      1  18040.png
2        18650  0.5000      1  18650.png
3        19161  0.2500      1  19161.png
4        18281  0.2500      1  18281.png
...        ...     ...    ...        ...
1357     18592  0.3750      1  18592.png
1358     18165  0.1875      1  18165.png
1359     19264  0.6875      1  19264.png
1360     18405  0.1875      1  18405.png
1361     18458  0.5000      1  18458.png

[1362 rows x 4 columns]


In [22]:
batch_size = 32
img_size = [240, 320]
shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift
)

angle_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/new_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

angle_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/new_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    shuffle=False,
    class_mode='other')


Found 817 validated image filenames.
Found 272 validated image filenames.
Found 273 validated image filenames.


In [23]:

angle_model = load_model("angle-model.h5")

angle_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

In [24]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=100
)
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

Epoch 1/100


2023-03-30 21:15:35.622126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-30 21:15:37.192246: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-03-30 21:15:39.064207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-30 21:15:39.097990: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f57b4b5b7e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-30 21:15:39.098039: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-03-30 21:15:39.143470: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-30 21:15:39.43

26/26 [==============================] - 20s 431ms/step - loss: 0.0219 - val_loss: 0.0224
Epoch 2/100
26/26 [==============================] - 9s 361ms/step - loss: 0.0062 - val_loss: 0.0076
Epoch 3/100
26/26 [==============================] - 9s 359ms/step - loss: 0.0055 - val_loss: 0.0095
Epoch 4/100
26/26 [==============================] - 9s 359ms/step - loss: 0.0053 - val_loss: 0.0069
Epoch 5/100
26/26 [==============================] - 10s 380ms/step - loss: 0.0045 - val_loss: 0.0052
Epoch 6/100
26/26 [==============================] - 10s 382ms/step - loss: 0.0045 - val_loss: 0.0045
Epoch 7/100
26/26 [==============================] - 10s 399ms/step - loss: 0.0048 - val_loss: 0.0060
Epoch 8/100
26/26 [==============================] - 10s 398ms/step - loss: 0.0047 - val_loss: 0.0045
Epoch 9/100
26/26 [==============================] - 10s 371ms/step - loss: 0.0042 - val_loss: 0.0043
Epoch 10/100
26/26 [==============================] - 9s 358ms/step - loss: 0.0042 - val_loss: 0.

In [38]:
angle_model1 = load_model("angle-model.h5")
angle_model = load_model("best-angle-model.h5")

test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)
test_loss = angle_model1.evaluate(
    angle_eval_generator,
    verbose=1
)

93/93 [==============================] - 8s 76ms/step - loss: 0.0059


In [2]:
angle_model = load_model("angle-model-fine-tuning.h5")
angle_model.summary()

2023-03-31 11:43:14.787757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 11:43:14.789762: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 11:43:14.789808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 11:43:14.789829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 4, 128)         1474688   
                                                                 
 batch_normalization_8 (Batc  (None, 3, 4, 128)        512       
 hNormalization)                                                 
                                                                 
 flatten_6 (Flatten)         (None, 1536)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               196736    
                                                                 
 batch_normalization_9 (Batc  (None, 128)             